In [1]:
import pandas as pd
import datetime
import requests
import warnings
warnings.filterwarnings('ignore')
import time
import urllib
from breeze_connect import BreezeConnect # breeze api
import os
import json
import webbrowser

In [2]:
universe = {'NIFTY':'NIFTY', 
            'BANKNIFTY':'CNXBAN', 
            'FINNIFTY':'NIFFIN', 
            'MIDCPNIFTY':'NIFSEL'}

In [3]:

###################################################################################################################################################################
def GET_REALTIME_DATA(symbol):
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36',}
    logs = []
    data=[]
    with requests.session() as req:
        try:
            req.get(f'https://www.nseindia.com/get-quotes/derivatives?symbol={symbol}',headers = headers)

            response = req.get(f'https://www.nseindia.com/api/quote-derivative?symbol={symbol}',headers = headers).json()
            ########################################################################################################
            data = []
            for i in range(len(response['stocks'])):
                identifier = response['stocks'][i]['metadata']['identifier']
                instrumentType = response['stocks'][i]['metadata']['instrumentType']
                expiryDate = response['stocks'][i]['metadata']['expiryDate']
                optionType = response['stocks'][i]['metadata']['optionType']
                strikePrice = response['stocks'][i]['metadata']['strikePrice']
                lastPrice = response['stocks'][i]['metadata']['lastPrice']
                numberOfContractsTraded = response['stocks'][i]['metadata']['numberOfContractsTraded']
                data.append([identifier, instrumentType, expiryDate, optionType, strikePrice, lastPrice, numberOfContractsTraded])
            data = pd.DataFrame(data, columns = ['Identifier', 'Instrument_Type', 'Expiry_Date', 'Option_Type', 'Strike', 'Last', 'Contracts_Traded'])
            expiry = []
            for i in range(len(data)):
                expiry.append(datetime.datetime.strptime(data['Expiry_Date'][i], '%d-%b-%Y').date())
            data['Expiry_Date'] = expiry

            futures_data = data[data['Instrument_Type'] == 'Index Futures']
            options_data = data[data['Instrument_Type'] == 'Index Options']

            futures_data.sort_values('Expiry_Date', inplace = True)
            options_data.sort_values('Expiry_Date', inplace = True)
        except:
            time.sleep(5)
            print(f'Fetching Failed {symbol} Trying Again ...')
            GET_REALTIME_DATA(symbol)
            
    futures_data.reset_index(inplace = True)
    futures_data = futures_data.drop(['index'], axis = 1)
    
    options_data.reset_index(inplace = True)
    options_data = options_data.drop(['index'], axis = 1)
    return futures_data, options_data
###################################################################################################################################################################
def GETKEYS(symbol, range_of_strikes, shortname):
    
    
    previous_close = GET_REALTIME_DATA(symbol)[0]['Last'].values[0]
    strikes = list(set(GET_REALTIME_DATA(symbol)[1]['Strike']))
    strikes.sort()
    strike_difference = strikes[int(len(strikes)/2) + 1] - strikes[int(len(strikes)/2)]
    rounded_close = round(previous_close/strike_difference) * strike_difference
    upper_range = rounded_close + range_of_strikes
    lower_range = rounded_close - range_of_strikes
    upper_range_strikes = [x for x in range(rounded_close, upper_range, strike_difference)]
    lower_range_strikes = [x for x in range(lower_range, rounded_close, strike_difference)]
    strikes = upper_range_strikes + lower_range_strikes
    strikes.sort()
    
    foldername = datetime.datetime.strftime(datetime.datetime.now(), '%Y-%m-%d')
    filename = 'FONSEScripMaster.csv'
    path = 'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/ScriptMaster'
    script_master_location = path + '/' + foldername + '/' + filename
    script_master = pd.read_csv(script_master_location)
    
    
    custom = script_master[script_master['ShortName'] == shortname]
    custom.reset_index(inplace = True)
    custom.drop('index', axis = 1, inplace = True)
    custom['ExpiryDate'] = pd.to_datetime(custom['ExpiryDate'])
    keys = []
    for i in range(len(custom)):
        exp = datetime.datetime.strftime(custom['ExpiryDate'][i], '%d_%b_%Y')
        strike_price = custom['StrikePrice'][i]
        if strike_price == 0:
            strike_price = ''
        else:
            strike_price = str(int(strike_price))
        key = custom['ShortName'][i] + custom['Series'][i][:3] + '_' + exp + '_' + strike_price + custom['OptionType'][i]
        keys.append(key)
    custom['Keys'] = keys
    custom_df = []
    for i in range(len(strikes)):
        strike = strikes[i]
        df = custom[custom['StrikePrice'] == strike]
        custom_df.append(df)
    custom_df = pd.concat(custom_df)
    custom_df.reset_index(inplace = True)
    custom_df.drop('index', axis = 1, inplace = True)
    
    token = {}
    for i in range(len(custom_df)):
        token[custom_df['Keys'][i]] = custom_df['Token'][i]
        
    future = custom[custom['Series'] == 'FUTURE']
    future.sort_values('ExpiryDate', inplace = True)
    future.reset_index(inplace = True)
    future.drop('index', axis = 1, inplace = True)
    
    for i in range(len(future)):
        token[future['Keys'][i]] = future['Token'][i]
    return token
###################################################################################################################################################################
timestamp_dict = {}
def on_ticks(ticks):
    try:

        universe = {'NIFTY 50' : 'NIFTY', 
                    'NIFTY BANK' : 'CNXBAN', 
                    'NIFTY FINANCIAL SERVICES INDEX' : 'NIFFIN', 
                    'NIFTY MIDCAP SELECT':'NIFSEL'}

        timestamp = datetime.datetime.now()
        timestamp = datetime.datetime.strftime(timestamp, '%Y-%m-%d %H:%M:%S.%f')

        # OHLC
        ltt = ticks['ltt']
        open = ticks['open']
        high = ticks['high']
        low = ticks['low']
        close = ticks['close']
        last = ticks['last']
        change = ticks['change']
        avgPrice = ticks['avgPrice']

        # Volume
        totalBuyQt = ticks['totalBuyQt']
        totalSellQ = ticks['totalSellQ']
        ttq = ticks['ttq']
        ttv = ticks['ttv']

        # 2nd level data
        bPrice = ticks['bPrice']
        bQty = ticks['bQty']
        sPrice = ticks['sPrice']
        sQty = ticks['sQty']
        ltq = ticks['ltq']

        # OI
        OI = ticks['OI']
        CHNGOI = ticks['CHNGOI']

        # Identity
        symbol = ticks['symbol']
        stock_name = ticks['stock_name']
        shorname = universe[stock_name]
        product_type = ticks['product_type'].upper()[:3]
        expiry_date = datetime.datetime.strptime(ticks['expiry_date'], '%d-%b-%Y')
        expiry_date = datetime.datetime.strftime(expiry_date, '%d_%b_%Y')
        try:
            strike_price = ticks['strike_price']
            right = ticks['right']
            if right == 'Call':
                right = 'CE'
            else:
                right = 'PE'
        except:
            strike_price = ''
            right = 'XX'
        key = shorname + product_type + '_' + expiry_date + '_' + strike_price + right

        row = {
               'shortname':shorname,
               'product_type':product_type,
               'expiry_date':expiry_date,
               'strike_price':strike_price,
               'right':right,
               'ltt': ltt, 
               'open': open, 
               'high':high, 
               'low':low, 
               'close':close, 
                'last':last,
               'change':change,
               'avgPrice' : avgPrice, 
               'ttv':ttv, 
               'oi':OI, 
               'bPrice':bPrice, 
               'bQty':bQty, 
               'sPrice':sPrice, 
               'sQty': sQty, 
               'lastPrice':last, 
               'ltq':ltq, 
               'totalBuyQt':totalBuyQt, 
               'totalSellQ':totalSellQ, 
               'ttq':ttq, 'key':key}
        
        timestamp_dict[timestamp] = row
    except:
        pass

In [4]:
token_dictionary = {}
for symbol in universe:
    if symbol in ['NIFTY', 'FINNIFTY'] :
        range_of_strikes = 500
    elif symbol in ['MIDCPNIFTY']:
        range_of_strikes = 250
    else:
        range_of_strikes = 1200
    shortname = universe [symbol]
    token_dictionary[symbol] = GETKEYS(symbol, range_of_strikes, shortname)

In [5]:
list_of_all_tokens = []
for i in token_dictionary:
    token_values = list(token_dictionary[i].values())
    for j in range(len(token_values)):
        list_of_all_tokens.append('4.1!' + str(token_values[j]))

In [6]:
api_key = '95n(71490#O^435820639Ao830~D666w'
api_secret = '%5d16s70=7nr5x6873z45932C89F*3sU'
authentication_url = "https://api.icicidirect.com/apiuser/login?api_key="+urllib.parse.quote_plus(api_key)
webbrowser.open(authentication_url)

True

In [7]:
api_session = '36933288'

In [8]:
# Future
api = BreezeConnect(api_key=api_key)
api.generate_session(api_secret=api_secret,session_token=api_session)
api.ws_connect()
api.on_ticks = on_ticks

In [9]:
def TickDataDownload(timestamp_dict):
    if len(timestamp_dict) >= 10000:
        timestamp_dict = {}
    else:
        current_minute = datetime.datetime.now().replace(second = 0, microsecond = 0)
        minute = datetime.datetime.strftime(current_minute, '%Y-%m-%d %H-%M')
        next_minute = current_minute + datetime.timedelta(minutes = 1)
        current_minute = datetime.datetime.strftime(current_minute, '%Y-%m-%d %H:%M:%S')
        next_minute = datetime.datetime.strftime(next_minute, '%Y-%m-%d %H:%M:%S')
        filename = 'Tick Data' + '_' + minute + '.pickle'
        file_path = 'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/' + filename
        df = pd.DataFrame(timestamp_dict).transpose().loc[current_minute:next_minute]
        df.to_pickle(file_path)
#         print('File Created', datetime.datetime.now())

In [10]:
api.subscribe_feeds(stock_token = list_of_all_tokens)

{'message': "Stock ['4.1!38940', '4.1!38941', '4.1!43016', '4.1!43017', '4.1!45717', '4.1!45718', '4.1!46439', '4.1!46448', '4.1!56881', '4.1!56882', '4.1!67293', '4.1!67294', '4.1!68041', '4.1!68042', '4.1!38942', '4.1!38943', '4.1!43550', '4.1!43850', '4.1!45719', '4.1!45720', '4.1!46449', '4.1!46450', '4.1!56883', '4.1!56884', '4.1!67295', '4.1!67296', '4.1!68045', '4.1!68046', '4.1!38944', '4.1!38945', '4.1!43851', '4.1!43852', '4.1!45721', '4.1!45722', '4.1!46456', '4.1!46457', '4.1!56885', '4.1!56886', '4.1!67297', '4.1!67298', '4.1!68047', '4.1!68050', '4.1!38948', '4.1!38949', '4.1!43853', '4.1!43854', '4.1!45723', '4.1!45728', '4.1!46458', '4.1!46459', '4.1!56887', '4.1!56888', '4.1!67299', '4.1!67300', '4.1!68051', '4.1!68055', '4.1!38950', '4.1!38951', '4.1!43857', '4.1!43858', '4.1!45729', '4.1!45730', '4.1!46460', '4.1!46461', '4.1!56889', '4.1!56890', '4.1!67301', '4.1!67302', '4.1!68056', '4.1!68057', '4.1!38952', '4.1!38953', '4.1!43859', '4.1!43866', '4.1!45731', '4.1!

In [11]:
today = datetime.datetime.now()
today = datetime.datetime.strftime(today, '%Y-%m-%d')
start = today + ' ' + '09:15:00'
end = today + ' ' + '15:30:00'
end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')

while end > datetime.datetime.now():
    current_minute = datetime.datetime.now().replace(second = 0, microsecond = 0)
    minute = datetime.datetime.strftime(current_minute, '%Y-%m-%d %H-%M')
    next_minute = current_minute + datetime.timedelta(minutes = 1)
    seconds_left = next_minute - current_minute
    seconds_left = seconds_left.seconds
    if len(timestamp_dict) >= 10000 and seconds_left >= 10:
        timestamp_dict = {}
    else:
        try:
#             print(len(timestamp_dict))
            TickDataDownload(timestamp_dict)
            time.sleep(0.05)
        except:
            print(len(timestamp_dict))
            print('Error Investigat', datetime.datetime.now())
            time.sleep(1)
    time.sleep(0.05)

6273
Error Investigat 2024-03-14 14:18:10.354484
6321
Error Investigat 2024-03-14 14:18:11.468812
6358
Error Investigat 2024-03-14 14:18:12.583460
6405
Error Investigat 2024-03-14 14:18:13.698649
6447
Error Investigat 2024-03-14 14:18:14.798333
6483
Error Investigat 2024-03-14 14:18:15.916695
6522
Error Investigat 2024-03-14 14:18:17.028759
6569
Error Investigat 2024-03-14 14:18:18.145099
6614
Error Investigat 2024-03-14 14:18:19.259911
6661
Error Investigat 2024-03-14 14:18:20.404989
6703
Error Investigat 2024-03-14 14:18:21.526529
6744
Error Investigat 2024-03-14 14:18:22.636860
6781
Error Investigat 2024-03-14 14:18:23.752374
6820
Error Investigat 2024-03-14 14:18:24.867627
6861
Error Investigat 2024-03-14 14:18:25.997045
6904
Error Investigat 2024-03-14 14:18:27.143923
6939
Error Investigat 2024-03-14 14:18:28.258599
6986
Error Investigat 2024-03-14 14:18:29.373774
7024
Error Investigat 2024-03-14 14:18:30.488532
7065
Error Investigat 2024-03-14 14:18:31.617626
7107
Error Investiga

In [ ]:
# # First Copy of the Dump
# current_hour = str(datetime.datetime.now().hour) #.minute
# filename = datetime.datetime.now()
# filename = datetime.datetime.strftime(filename, '%Y-%m-%d') + ' ' + current_hour
# file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
# copy = timestamp_dict.copy()
# with open(file_path, 'w') as json_file:
#     copy = pd.DataFrame(copy).transpose()
#     copy = copy.to_dict()
#     json.dump(copy, json_file)

In [ ]:
# today = datetime.datetime.now()
# today = datetime.datetime.strftime(today, '%Y-%m-%d')
# start = today + ' ' + '09:15:00'
# end = today + ' ' + '15:30:00'
# end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')

# while end > datetime.datetime.now():
#     try:
#         current_hour = str(datetime.datetime.now().minute)
#         filename = datetime.datetime.now()
#         filename = datetime.datetime.strftime(filename, '%Y-%m-%d') + ' ' + current_hour
#         file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
#         with open(file_path) as user_file:
#             old = user_file.read()
#         old = json.loads(old)
#         old = pd.DataFrame(old)
#         old_index = old.index[-1]
#         new = pd.DataFrame(timestamp_dict).transpose().loc[old_index:]
#         df = pd.concat([old, new])
#         dictionary = df.to_dict()
#         with open(file_path, 'w') as json_file:
#             print(f'File Generated {datetime.datetime.now()}')
#             json.dump(dictionary, json_file)
#         time.sleep(0.1)
#     except:
#         # First Copy of the Dump
#         current_hour = str(datetime.datetime.now().minute)
#         filename = datetime.datetime.now()
#         filename = datetime.datetime.strftime(filename, '%Y-%m-%d') + ' ' + current_hour
#         file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
#         copy = timestamp_dict.copy()
#         with open(file_path, 'w') as json_file:
#             copy = pd.DataFrame(copy).transpose()
#             copy = copy.to_dict()
#             json.dump(copy, json_file)

In [ ]:
# # Assuming timestamp_dict is initialized elsewhere in your code
# timestamp_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}

# # Create a lock
# dict_lock = threading.Lock()

# today = datetime.datetime.now()
# today = datetime.datetime.strftime(today, '%Y-%m-%d')
# start = today + ' ' + '09:15:00'
# end = today + ' ' + '15:30:00'
# end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')

# while end > datetime.datetime.now():
#     filename = datetime.datetime.now()
#     filename = datetime.datetime.strftime(filename, '%Y-%m-%d')
    
#     # Acquire the lock before copying the dictionary
#     with dict_lock:
#         dict_copy = copy.deepcopy(timestamp_dict)
    
#     file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
    
#     print(f'File Generated {datetime.datetime.now()}')
    
    with open(file_path, 'w') as json_file:
        json.dump(dict_copy, json_file)
    
#     time.sleep(0.1)


In [ ]:
# import json
# import datetime
# import time
# import copy

# today = datetime.datetime.now()
# today = datetime.datetime.strftime(today, '%Y-%m-%d')
# start = today + ' ' + '09:15:00'
# end = today + ' ' + '15:30:00'
# end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')

# while end > datetime.datetime.now():
#     filename = datetime.datetime.now()
#     filename = datetime.datetime.strftime(filename, '%Y-%m-%d')
    
#     # Deep copy the dictionary to avoid "dictionary changed size during iteration" error
#     dict_copy = copy.deepcopy(timestamp_dict)
    
#     file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
    
#     print(f'File Generated {datetime.datetime.now()}')
    
#     with open(file_path, 'w') as json_file:
#         json.dump(dict_copy, json_file)
    
#     time.sleep(0.1)


In [ ]:
# today = datetime.datetime.now()
# today = datetime.datetime.strftime(today, '%Y-%m-%d')
# start = today + ' ' + '09:15:00'
# end = today + ' ' + '15:30:00'
# end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
# while end > datetime.datetime.now():
#     filename = datetime.datetime.now()
#     filename = datetime.datetime.strftime(filename, '%Y-%m-%d')
#     file_path = f'C:/Users/dhawalsheth/Desktop/Revised Files/Intraday Data/Tick Data/Tick Data {filename}.json'
#     dict_copy  = timestamp_dict
#     print(f'File Generated {datetime.datetime.now()}')
#     with open(file_path, 'w') as json_file:
#         json.dump(dict_copy, json_file)
#     time.sleep(0.1)

In [ ]:
# today = datetime.datetime.now()
# today = datetime.datetime.strftime(today, '%Y-%m-%d')
# start = today + ' ' + '09:15:00'
# end = today + ' ' + '15:30:00'
# end = datetime.datetime.strptime(end, '%Y-%m-%d %H:%M:%S')
# while end > datetime.datetime.now():
#     filename = datetime.datetime.now()
#     filename = datetime.datetime.strftime(filename, '%Y-%m-%d')
#     print(f'File Generated {datetime.datetime.now()}')
#     df = pd.DataFrame(timestamp_dict).transpose()
#     df.to_pickle(f'Tick Data {filename}.pickle')
#     time.sleep(0.2)

In [ ]:
datetime.datetime.now() + datetime.timedelta(minutes = 1)

In [ ]:
datetime.datetime.now().minute  + 1

In [ ]:
datetime.timedelta(days =1)

In [ ]:
current_minute = datetime.datetime.now().replace(second = 0, microsecond = 0)
next_minute = current_minute + datetime.timedelta(minutes = 1)

In [ ]:
datetime.datetime.strftime(current_minute, '%Y-%m-%d %H:%M:%S')

In [ ]:
next_minute

In [ ]:
time.sleep(0.05)